In [1]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

# Add config.json file to the workspace
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential, path="config.json")

Found the config file in: /mnt/batch/tasks/shared/LS_root/mounts/clusters/jcharley4/code/Users/jcharley/config.json


## Create Online Endpoint

In [7]:
from azure.ai.ml.entities import ManagedOnlineEndpoint

# create an online endpoint
online_endpoint = ManagedOnlineEndpoint(
    name="taxi-online-endpoint-ju1", 
    description="Taxi online endpoint",
    auth_mode="aml_token",
)
ml_client.online_endpoints.begin_create_or_update(
    online_endpoint,   
)


## 2. Create Online Deployment

To create a deployment to online endpoint, you need to specify the following elements:

* Model files (or specify a registered model in your workspace)
* Scoring script - code needed to do scoring/inferencing
* Environment - a Docker image with Conda dependencies, or a dockerfile
* Compute instance & scale settings

Note that if you're deploying **MLFlow models**, there's no need to provide **a scoring script** and execution **environment**, as both are autogenerated.

We can create an **online deployment** with cli v2 or sdk v2 using the following syntax:


In [5]:
# create online deployment
from azure.ai.ml.entities import ManagedOnlineDeployment, Model, Environment

model = "taxi-model@latest"

blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name="taxi-online-endpoint-ju1",
    model=model,
    instance_type="Standard_DS2_v2",
    instance_count=1,
)

ml_client.online_deployments.begin_create_or_update(
    deployment=blue_deployment
)


Check: endpoint credit-endpoint-80e2bf22 exists


HttpResponseError: (BadRequest) The request is invalid.
Code: BadRequest
Message: The request is invalid.
Exception Details:	(InvalidOverwriteRequest) Invalid overwrite request - cannot update location, deployment type, sku, or os type. Please delete and redeploy this service.
	Code: InvalidOverwriteRequest
	Message: Invalid overwrite request - cannot update location, deployment type, sku, or os type. Please delete and redeploy this service.
Additional Information:Type: ComponentName
Info: {
    "value": "managementfrontend"
}Type: Correlation
Info: {
    "value": {
        "operation": "89a2016fd15b03d2c994a8ec927c2784",
        "request": "c9b9fdba2f34b801"
    }
}Type: Environment
Info: {
    "value": "westeurope"
}Type: Location
Info: {
    "value": "westeurope"
}Type: Time
Info: {
    "value": "2022-12-30T15:23:17.6607866+00:00"
}